In [33]:
### import required libraries
import requests
import time
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [34]:
### set date range for weekly report
date_range_high = datetime.today().date()
date_range_low = datetime.today().date() - timedelta(days=7)

In [35]:
user_agent = 'Mozilla/5.0 (iPad; U; CPU OS 3_2_1 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Mobile/7B405'

#### Reuters M&A News

In [36]:
### webscrape Reuters M&A News
date_range_high = datetime.today().date()
date_range_low = datetime.today().date() - timedelta(days=7)
user_agent = 'Mozilla/5.0 (iPad; U; CPU OS 3_2_1 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Mobile/7B405'

date_filter = date_range_high
page_number = 1
reuters_news_dict = {}
reuters_news_url = "https://www.reuters.com/news/archive/mergersnews?view=page&page={}&pageSize=10"

while date_filter >= date_range_low:

    reuters_raw = requests.get(reuters_news_url.format(page_number), headers={'User-Agent': user_agent})
    reuters_bs4 = BeautifulSoup(reuters_raw.content, features="lxml")

    reuters_news_block = reuters_bs4.find_all('div', class_ = 'column1 col col-10')
    reuters_news_list  = reuters_news_block[0].find_all('h3', class_ = 'story-title')
    reuters_timestamp_list = reuters_news_block[0].find_all('span', class_ = 'timestamp')

    link_list = [i.find('a')['href'] for i in reuters_news_block[0].find_all('div', class_ = 'story-content')]
    link_list = ['reuters.com'+ i for i in link_list]

    for i in range(len(reuters_news_list)):
        title = reuters_news_list[i].get_text()
        title = title.split('\n\t\t\t\t\t\t\t\t')[1]

        a_date = reuters_timestamp_list[i].get_text()
        if 'am' in a_date or 'pm' in a_date:
            a_date = datetime.today().date()
        else:
            a_date = datetime.strptime(a_date, "%b %d %Y").date()

        reuters_news_dict[title] = [a_date, link_list[i]]

    date_filter = a_date
    page_number += 1


In [37]:
reuters_news_dict = {title: info[1] for title, info in reuters_news_dict.items() if info[0] >= date_range_low}

In [38]:
reuters_news_dict 

{'UPDATE 2-Pembina, KKR ink deals to boost western Canada gas infrastructure': 'reuters.com/article/pembina-pipe-kkr-joint-venture/update-2-pembina-kkr-ink-deals-to-boost-western-canada-gas-infrastructure-idUSL4N2V43GS',
 'Ukraine crisis poses new risks for Alitalia successor ITA - chairman': 'reuters.com/article/ita-airways/ukraine-crisis-poses-new-risks-for-alitalia-successor-ita-chairman-idUSL8N2V44SL',
 "UPDATE 2-FTC prepares for possible challenge to Amazon's $8.5 bln MGM deal - The Information": 'reuters.com/article/usa-ftc-antitrust-amazoncom/update-2-ftc-prepares-for-possible-challenge-to-amazons-8-5-bln-mgm-deal-the-information-idUSL4N2V40ML',
 "U.S. FTC preparing for possible challenge to Amazon's $8.5 bln MGM deal - The Information": 'reuters.com/article/usa-ftc-antitrust-amazoncom/u-s-ftc-preparing-for-possible-challenge-to-amazons-8-5-bln-mgm-deal-the-information-idUSL4N2V4090',
 "UPDATE 1-'The new Atos' is on the way to recovery in 2022, CEO says": 'reuters.com/article/at

#### WSJ Deals News

In [39]:
def scrape_wsj_news(event, context):

    date_range_high = datetime.today().date()
    date_range_low = datetime.today().date() - timedelta(days=7)

    user_agent = 'Mozilla/5.0 (iPad; U; CPU OS 3_2_1 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Mobile/7B405'
    wsj_deals_url = "https://www.wsj.com/news/types/deals-deal-makers?page={}"
    page_number = 1
    date_filter = date_range_high

    title_list = []
    date_list = []
    link_list = []

    while date_filter >= date_range_low:
        wsj_raw = requests.get(wsj_deals_url.format(page_number), headers={'User-Agent': user_agent})
        status = True
        timer = 0

        while status and timer <=9:
            if wsj_raw.status_code != 200:
                print("wsj page {} status code is not 200, entering sleep for 3 seconds".format(page_number))
                time.sleep(3)
                timer += 3
                wsj_raw = requests.get(wsj_deals_url.format(page_number), headers={'User-Agent': user_agent})
            else:
                status = False
        
        if wsj_raw.status_code == 200:

            wsj_bs4 = BeautifulSoup(wsj_raw.content, features="lxml")
            article_list = wsj_bs4.select('h2[class*="headline"]')
            combined_ts = wsj_bs4.select('div[class*="timestamp"]')

            if article_list!=[] and combined_ts!=[]:

                for article in article_list:
                    content = article.get_text()
                    link = article.find('a')['href']
                    title_list.append(content)
                    link_list.append(link)
                    
                for timestamp in combined_ts:
                    try:
                        a_date = timestamp.find('div').get_text()
                        a_date = datetime.strptime(a_date, "%B %d, %Y").date()
                        date_list.append(a_date)
                    except:
                        pass
            else:
                a_date = datetime.today().date() - timedelta(days=8)

            date_filter = a_date
            page_number += 1
        
        else:
            date_filter = datetime.today().date() - timedelta(days=8)

    wsj_news_dict = dict(zip(title_list, zip(date_list, link_list)))
    wsj_news_dict = {title: info[1] for title, info in wsj_news_dict.items() if info[0] >= date_range_low}

    return wsj_news_dict

In [40]:
scrape_wsj_news(None, None)

{'Zendesk Scraps Deal With SurveyMonkey Parent': 'https://www.wsj.com/articles/zendesk-terminates-deal-with-surveymonkey-parent-11645830504',
 'Healthcare Realty Trust Nears Deal With Healthcare Trust of America': 'https://www.wsj.com/articles/healthcare-realty-trust-nears-deal-with-healthcare-trust-of-america-11645741490',
 'Standard General to Buy TV Broadcaster Tegna for $5.4 Billion': 'https://www.wsj.com/articles/standard-general-to-buy-tv-broadcaster-tegna-for-5-4-billion-11645548301',
 'SoFi to Buy Technisys for About $1.1 Billion': 'https://www.wsj.com/articles/sofi-to-buy-banking-infrastructure-firm-technisys-for-about-1-1-billion-11645524001'}

#### New York Times Mergers News

In [41]:
def scrape_nyt_news(event, context):
    date_range_low = datetime.today().date() - timedelta(days=7)
    user_agent = 'Mozilla/5.0 (iPad; U; CPU OS 3_2_1 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Mobile/7B405'
    ma_news_url = "https://www.nytimes.com/topic/subject/mergers-acquisitions-and-divestitures"
    nyt_raw = requests.get(ma_news_url, headers={'User-Agent': user_agent})

    status = True
    timer = 0

    while status and timer <=9:
        if nyt_raw.status_code != 200:
            print("nyt status code is not 200, entering sleep for 3 seconds")
            time.sleep(3)
            timer += 3
            nyt_raw = requests.get(ma_news_url, headers={'User-Agent': user_agent})
        else:
            status = False

    title_list = []
    date_list = []
    link_list = []

    if nyt_raw.status_code == 200:
        nyt_content = BeautifulSoup(nyt_raw.content)

        title_list = nyt_content.find_all('ol')[0].find_all('h2')
        link_list = nyt_content.find_all('ol')[0].find_all('a')

        if title_list !=[] and link_list !=[]:
            try:
                title_list = [i.get_text() for i in title_list]
                temp_list = [i['href'].split('/')[1:4] for i in link_list]
                link_list = ["nytimes.com"+i['href'] for i in link_list]
                
                for sub in temp_list:
                    date_elements = [int(i) for i in sub]
                    a_date = datetime(year = date_elements[0], month = date_elements[1], day = date_elements[2]).date()
                    date_list.append(a_date)
            except:
                title_list = []
                date_list = []
                link_list = []
        else:
            title_list = []
            date_list = []
            link_list = []

    nyt_ma_news_dict = dict(zip(title_list, zip(date_list,link_list)))
    nyt_ma_news_dict = {title: info[1] for title, info in nyt_ma_news_dict.items() if info[0] >= date_range_low}

    return nyt_ma_news_dict

In [42]:
scrape_nyt_news(None, None)

{'Shell Quits Russia, Joining BP as Oil Giants Object to Attack on Ukraine': 'nytimes.com/2022/02/28/business/russia-oil-companies.html',
 'Uber distances itself from Yandex.Taxi, the Russian ride-sharing service.': 'nytimes.com/2022/02/28/business/russia-uber-yandex-taxi.html',
 'Neil Diamond Sells Entire Catalog to Universal Music': 'nytimes.com/2022/02/28/arts/music/neil-diamond-catalog-sale.html',
 'Chris Licht, a Creator of ‘Morning Joe’ and ‘Colbert’ Producer, Is Set to Run CNN': 'nytimes.com/2022/02/26/business/media/cnn-chris-licht.html',
 'Justice Dept. Sues to Block $13 Billion Deal by UnitedHealth Group': 'nytimes.com/2022/02/24/business/doj-antitrust-lawsuit-unitedhealth.html',
 'Met Buys Italian Renaissance Bronze After Two Decades on the Hunt': 'nytimes.com/2022/02/23/arts/design/met-italian-renaissance-bronze-cavalli.html',
 'Volkswagen is considering a plan to spin off Porsche in a public offering.': 'nytimes.com/2022/02/22/automobiles/volkswagen-porsche-ipo.html'}

In [4]:
s = Service('chromedriver/chromedriver')
driver = webdriver.Chrome(service= s)
str1 = driver.capabilities['browserVersion']
str2 = driver.capabilities['chrome']['chromedriverVersion'].split(' ')[0]
print("browser version", str1)
print("chrome driver version", str2)
if str1[0:2] != str2[0:2]: 
  print("please download correct chromedriver version")

browser version 98.0.4758.109
chrome driver version 97.0.4692.71
please download correct chromedriver version


In [1]:
options = Options()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
s = Service('chromedriver/chromedriver')
driver = webdriver.Chrome(service= s)

#### theMiddleMarket M&A News

In [2]:
ma_news_url = 'https://www.themiddlemarket.com/latest-news'
driver.get(ma_news_url)

latest_news = driver.find_elements(By.XPATH,"/html/body/main/div/div/div/div[1]/div/div")
latest_news = latest_news[0].text.split('\n')

ma_article_list = [i for count, i in enumerate(latest_news) if count%2 == 0 ] 
ma_date_list = [i for count, i in enumerate(latest_news) if count%2 != 0 ] 
theMiddleMarket_news_dict =  dict(zip(ma_article_list, ma_date_list))

In [ ]:
theMiddleMarket_news_dict = {key: datetime.strptime(value.title(), '%B %d, %Y').date() for key, value in theMiddleMarket_news_dict.items()}
theMiddleMarket_news_dict = {title: date for title, date in theMiddleMarket_news_dict.items() if date >= date_range_low}

In [ ]:
theMiddleMarket_news_dict

#### New York Times Mergers News

In [ ]:
nyt_url = "https://www.nytimes.com/topic/subject/mergers-acquisitions-and-divestitures"
driver.get(nyt_url)
nyt_news_raw = driver.find_elements(By.XPATH, '//*[@id="collection-Mergers, Acquisitions and Divestitures"]/div[1]/div')[0].text.split('\n')

In [ ]:
index_list = []
date_list = []
for count, text in enumerate(nyt_news_raw):
    try:
        a_date = datetime.strptime(text, "%b. %d, %Y")
        date_list.append(a_date)
        index_list.append(count -2)
    except:
        pass
    
title_list = [nyt_news_raw[i] for i in index_list]
nyt_ma_news_dict = dict(zip(title_list, date_list))

In [ ]:
driver.close()

#### Seeking Alpha M&A News

In [ ]:
article_list = []
formatted_date_list = []

driver.get('https://seekingalpha.com/market-news/m-a?page=1')

article_block = driver.find_elements(By.XPATH, "//*[@id='content']/div/div[2]/div/div[2]/section/div/div/div/div[2]")[0].text.split('\n')

for i, j in enumerate(article_block):
    if i%2 == 0:
        article_list.append(j)

date_list = driver.find_elements(By.XPATH, "//*[@id='content']/div/div[3]/div/div[2]/section/div/div/div/div[2]/article[1]/div/div/footer/span[2]")
date_list = [i.text for i in date_list] 

for a_date in date_list:
    if 'Today' in a_date:
        a_date = datetime.today().date()
    elif 'Yesterday' in a_date:
        a_date = datetime.today().date() - timedelta(days = 1)
    elif len(a_date.split(' ')) == 3:
        a_date = a_date + ' ' + str(datetime.today().date().year)
        a_date = datetime.strptime(a_date, "%a, %b. %d %Y").date()
    else:
        a_date = datetime.strptime(a_date, "%a, %b. %d, %Y").date()
    
    formatted_date_list.append(a_date)

In [ ]:
date_list

In [ ]:
seeking_alpha_news_dict = dict(zip(article_list, formatted_date_list))

In [ ]:
seeking_alpha_news_dict